# Transformers for BoolQ reading comprehension

## Sources

My sources for this project are linked in the respecting sections of the notebook. I used AI tools such as ChatGPT to correct my writing and grammar in stage 1 of this project and plan on using it for debugging during stage 2.

## Setup

### **Importing Python Packages**
Making sure the notebook is reproducible and runs without error, I will install the necessary libraries in a pip cell below.

### **Data Loading and Split**
The BoolQ dataset contains binary question-answer pairs. Each entry consists of a question, a passage, and the corresponding binary answer (yes/no). The dataset is split as required by the course materials:
- **Train Split:** The first 8427 entries of the training data.
- **Validation Split:** The last 1000 entries of the training data.
- **Test Split:** The validation split provided in the BoolQ dataset (3270 entries).

### **Seeding for Reproducibility**
A seed value of 42 is used to ensure reproducibility of results across different runs.

### **Batch size**
Setting the batch size in the beginning of the notebook for use throughout the code.

In [55]:
# TODO: pip install necessary packages
# %pip install 

In [56]:
import wandb

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel, get_scheduler
from torch.utils.data import DataLoader, Dataset
from typing import Any, Dict
import torch
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from pathlib import Path
from torchmetrics.classification import BinaryAccuracy, BinaryConfusionMatrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
from sklearn.metrics import confusion_matrix


In [57]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cpu


In [58]:
pl.seed_everything(42, workers=True)

Seed set to 42


42

In [59]:
BATCH_SIZE: int = 32
MAX_LENGTH: int = 512

print(f"Batch_size: {BATCH_SIZE}"
      f"\nMax_length: {MAX_LENGTH}")

Batch_size: 32
Max_length: 512


## Preprocessing

### Tokenizer
In this project, the `AutoTokenizer` from the Hugging Face Transformers library is used to handle text preprocessing. Specifically, the `bert-base-cased` associated with the "bert-base-cased" model is instantiated. This tokenizer manages:
- Whitespace and special character removal.
- Retaining case sensitivity to preserve semantic meaning.
- Padding and truncation to ensure sequences are within the 512-token limit.

##### Tokenization Details
- **Questions:** The length of questions is capped at 21 tokens, based on dataset analysis.
- **Passages:** Passages are padded or truncated to ensure the overall sequence remains within the 512-token limit.


#### Handling OOV words
The BERT Tokenizer utilizes a WordPiece tokenization strategy. Words not present in the vocabulary are decomposed into subword units. For example, an unknown word like "unhappiness" might be tokenized into "un", "##happiness". This approach ensures that even unseen words are represented through known subword components.


#### Padding and Truncation
Sequences are padded to a maximum length of 512 tokens, aligning with BERT's architecture constraints. Padding is applied to the maximum sequence length, not just the length of the longest sequence in a batch. This approach ensures consistent input dimensions across all batches, prioritizing simplicity and uniformity over computational efficiency. While dynamic batch padding could improve efficiency and reduce memory usage, the focus of this project is not on optimizing transformer models for efficiency. Therefore, fixed padding is used to streamline implementation and maintain uniformity.


#### Case Sensitivity
The "bert-base-cased" tokenizer is case-sensitive, distinguishing between words like "Apple" and "apple". This sensitivity preserves the semantic nuances of the text.


### Embedding Layer
The model leverages the pre-trained embeddings form the "bert-base-cased" model. These embeddings are fine-tuned during training to adapt to the specific nuances of the BoolQ dataset.


### Positional Embeddings
BERT inherently incorporates positional embeddings, unlike the transformer encoder in the last project, to capture the order of tokens in a sequence.


### Input / Output / Label Format
Each data point consists of a question, passage, and binary label (True/False). The preprocessing transforms these into the following formats:
- **Tokenization**:
    - *input*: Concatenated question and passage with `[CLS]` at the start, and `[SEP]` is placed between and at the end of the concatenated sequence.
    - *output*: Tokenized sequences with attention masks:
        - `input_ids`: Tensor of shape `(batch_size, sequence_length)`, representing token IDs.
        - `attention_mask`: Tensor of shape `(batch_size, sequence_length)`, indicating non-padded tokens.
- **Transformer Encoder**:
    - *input*: Tokenized `input_ids` and `attention_mask` tensors with shape `(batch_size, sequence_length)`.
    - *output*: Contextualized embeddings from the final encoder layer of shape `(batch_size, sequence_length, hidden_dim)`.
- **Classifier**:
    - *input*: The embedding corresponding to the `[CLS]` token with shape `(batch_size, hidden_dim)`.
    - *output*: A single logit per data point with shape `(batch_size, 1)`, representing the probability of the label being True.
- **Label format**:
    - Binary labels are encoded as integers (`0` for False, `1` for True).

In [60]:
class BoolQDataset(Dataset):
    def __init__(self, data: Dict[str, Any], tokenizer: AutoTokenizer, max_length: int = MAX_LENGTH):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self) -> int:
        return len(self.data["question"])
    
    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        question = self.data["question"][idx]
        passage = self.data["passage"][idx]
        label = self.data["answer"][idx]
        
        # Tokenize sequences
        encoded = self.tokenizer(
            question,
            passage,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        )
        
        # Tokenizer test
        assert encoded["input_ids"].shape[-1] <= self.max_length, "Token length exceeds max_length!"
        assert encoded["input_ids"].shape == encoded["attention_mask"].shape, "Mismatched token shapes!"
        
        return {
            "input_ids": encoded["input_ids"].squeeze(0),  # Remove batch dimension
            "attention_mask": encoded["attention_mask"].squeeze(0),  # Remove batch dimension
            "label": torch.tensor(label, dtype=torch.float),  # Float for binary classification
        }

In [61]:
class BoolQDataModule(pl.LightningDataModule):
    def __init__(self, tokenizer_name: str, batch_size: int = BATCH_SIZE, max_length: int = MAX_LENGTH):
        super().__init__()
        self.tokenizer_name = tokenizer_name
        self.batch_size = batch_size
        self.max_length = max_length
        
    def prepare_data(self) -> None:
        # Loading the dataset based on lecture slides
        self.train_dataset = load_dataset('google/boolq', split='train[:-1000]')
        self.val_dataset = load_dataset('google/boolq', split='train[-1000:]')
        self.test_dataset = load_dataset('google/boolq', split='validation')
    
    def setup(self, stage: str = None) -> None:
        # Initialize Tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(self.tokenizer_name)
        
        # Create Datasets
        self.train_dataset = BoolQDataset(self.train_dataset, self.tokenizer, self.max_length)
        self.val_dataset = BoolQDataset(self.val_dataset, self.tokenizer, self.max_length)
        self.test_dataset = BoolQDataset(self.test_dataset, self.tokenizer, self.max_length)
        
        # Test dataset length
        assert len(self.train_dataset) == 8427, "Train dataset length is incorrect!"
        assert len(self.val_dataset) == 1000, "Validation dataset length is incorrect!"
        assert len(self.test_dataset) == 3270, "Test dataset length is incorrect!"
    
    def train_dataloader(self) -> DataLoader:
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)
    
    def val_dataloader(self) -> DataLoader:
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=True)
        
    def test_dataloader(self) -> DataLoader:
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=True)
        

In [62]:
data_module = BoolQDataModule(tokenizer_name="bert-base-cased", batch_size=BATCH_SIZE)

data_module.prepare_data()
data_module.setup()

for batch in data_module.train_dataloader():
    assert batch["input_ids"].shape[0] == BATCH_SIZE, "Batch size mismatch!"
    print(f"Batch loaded successfully with shape: {batch['input_ids'].shape}")
    break

Batch loaded successfully with shape: torch.Size([32, 512])


## Model

### Architecture
- **Pretrained Transformer Encoder:**
  - Hugging Face’s `bert-base-cased` processes tokenized inputs.
  - The output corresponding to the `[CLS]` token is extracted as a fixed-size representation.
  - *The `bert-base-cased` model is pretrained using a masked language modeling and next sentence prediction objective and not question answering. This is why I deem it usable for this project where fine-tuned pretrained models are not allowed.*
- **Classifier:**
  - A two-layer fully connected network processes the `[CLS]` token embedding:
    - **First Layer:** Projects the embedding to the hidden dimension with ReLU activation.
    - **Dropout Layer:** Introduced after ReLU to reduce overfitting.
    - **Second Layer:** Maps the hidden representation to a single binary output using Sigmoid activation.


### Loss Function
The Binary Cross-Entropy Loss (BCE) function is used to calculate the difference between predicted and true labels for binary classification.


### Optimizer
*Learning rates stated here are for testing model functionality. Hyperparameters for experiments stated in `Experiments`.*
- **AdamW Optimizer:**
  - A learning rate of `2e-5` is used for the Transformer encoder.
  - A higher learning rate of `2e-4` is applied to the classifier layers to allow faster convergence.


### Checkpointing and Early Stopping
- **Checkpointing:** Save the model with the best validation accuracy. Criteria for this will be the maximum validation accuracy.
- **Early Stopping:** Terminates training if validation loss does not improve for 10 consecutive epochs.


### Correctness Tests
- **Tokenization**:
    - Ensure the tokenized output does not exceed 512 tokens.
    - Verify alignment between `input_ids` and `attention_mask` dimensions.

- **DataLoader**:
    - Verify batch size consistency during data loading.
    - Check that the output tensors for `input_ids` and `attention_mask` match the expected batch size and sequence length.

- **Model Input/Output**:
    - Confirm the input to the Transformer encoder has the shape `(batch_size, sequence_length)`.
    - Validate that the output of the Transformer encoder has the shape `(batch_size, sequence_length, hidden_dim)`.

- **Classifier Dimensions**:
    - Check that the input to the classifier corresponds to the `[CLS]` token embedding with shape `(batch_size, hidden_dim)`.
    - Ensure the output of the classifier has the shape `(batch_size, 1)`.

- **Reproducibility**:
    - Validate consistent results across multiple runs with the same random seed.


In [63]:
class TransformerClassifier(pl.LightningModule):
    def __init__(
            self,
            model_name: str,
            learning_rate: float,
            hidden_dim: int,
            dropout_rate: float,
            warmup_steps: float = 0.1,  # in percent of total steps
            weight_decay: float = 0.0
    ):
        super(TransformerClassifier, self).__init__()
        self.save_hyperparameters()
        self.bert = AutoModel.from_pretrained(model_name)
        self.bert.train()
        
        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
        self.loss_fn = nn.BCELoss()
        
        # Metrics
        self.train_accuracy = BinaryAccuracy()
        self.val_accuracy = BinaryAccuracy()
        self.test_accuracy = BinaryAccuracy()
        self.val_conf_matrix = BinaryConfusionMatrix()
        self.test_conf_matrix = BinaryConfusionMatrix()
        
    def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_token = outputs.last_hidden_state[:, 0, :]  # CLS Token
        logits = self.classifier(cls_token)
        return logits.squeeze(-1)
    
    def training_step(self, batch: Dict[str, torch.Tensor], batch_idx: int) -> torch.Tensor:
        logits = self(batch["input_ids"], batch["attention_mask"])
        loss = self.loss_fn(logits, batch["label"])
        preds = (logits > 0.5).float()
        self.train_accuracy.update(preds, batch["label"])
        self.log("train_loss", loss)
        self.log("train_acc", self.train_accuracy.compute(), prog_bar=True)
        return loss
    
    def val_step(self, batch: Dict[str, torch.Tensor], batch_idx: int) -> Dict[str, Any]:
        logits = self(batch["input_ids"], batch["attention_mask"])
        loss = self.loss_fn(logits, batch["label"])
        preds = (logits > 0.5).float()
        acc = self.val_accuracy.update(preds, batch["label"])
        self.val_conf_matrix.update(preds, batch["label"])
        self.log("val_loss", loss, prog_bar=True)
        return {"val_loss": loss, "val_acc": acc}
    
    def on_validation_epoch_end(self) -> None:
        val_acc = self.val_accuracy.compute()
        cm = self.val_conf_matrix.compute().cpu().numpy()

        fig, ax = plt.subplots(figsize=(5, 5))
        ax.matshow(cm, cmap=plt.cm.Blues)
        for (i, j), val in np.ndenumerate(cm):
            ax.text(j, i, f"{val}", ha="center", va="center")
        plt.xlabel("Predicted Labels")
        plt.ylabel("True Labels")
        plt.title("Validation Confusion Matrix")
        plt.show()

        self.log('val_acc', val_acc, prog_bar=True)
        wandb.log({"val_confusion_matrix": wandb.Image(fig)})

        # Reset metrics
        self.val_accuracy.reset()
        self.val_conf_matrix.reset()
    
    def test_step(self, batch: Dict[str, torch.Tensor], batch_idx: int) -> None:
        logits = self(batch['input_ids'], batch['attention_mask'])
        loss = self.loss_fn(logits, batch['label'])
        preds = (logits > 0.5).float()
        acc = self.test_accuracy.update(preds, batch['label'])
        self.test_conf_matrix.update(preds, batch['label'])
        self.log('test_loss', loss, prog_bar=True)
        
        return {'test_loss': loss, 'test_acc': acc}
        
    def on_test_epoch_end(self) -> None:
        test_acc = self.test_accuracy.compute()
        cm = self.test_conf_matrix.compute().cpu().numpy()

        # Plot confusion matrix
        fig, ax = plt.subplots(figsize=(5, 5))
        ax.matshow(cm, cmap=plt.cm.Blues)
        for (i, j), val in np.ndenumerate(cm):
            ax.text(j, i, f"{val}", ha="center", va="center")
        plt.xlabel("Predicted Labels")
        plt.ylabel("True Labels")
        plt.title("Test Confusion Matrix")

        # Show the plot in the notebook
        plt.show()

        # Log metrics
        self.log('test_acc', test_acc, prog_bar=True)
        wandb.log({"test_confusion_matrix": wandb.Image(fig)})

        # Reset metrics
        self.test_accuracy.reset()
        self.test_conf_matrix.reset()
        
    def configure_optimizers(self) -> Dict[str, Any]:
        transformer_params = list(self.bert.parameters())
        classifier_params = list(self.classifier.parameters())
        
        # classifier lr is higher for faster convergence
        optimizer = torch.optim.AdamW(
            [
                {"params": transformer_params, "lr": self.hparams.learning_rate, "weight_decay": self.hparams.weight_decay},
                {"params": classifier_params, "lr": self.hparams.learning_rate * 10, "weight_decay": self.hparams.weight_decay},
            ]
        )
        
        total_steps = math.ceil(self.trainer.estimated_stepping_batches)
        warmup_steps = int(total_steps * self.hparams.warmup_steps)
        
        scheduler = get_scheduler(
            "linear",
            optimizer=optimizer,
            num_warmup_steps=warmup_steps,
            num_training_steps=total_steps,
        )
        
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "step",
                "frequency": 1,
            },
        }

## Experiments
**Batch Size:** I will start with a batch_size of 16 and increase it to the maximum my hardware can handle then leaving it fixed as it is not a hyperparameter.


### Hyperparameters
The following hyperparameter ranges were explored during tuning:
- **Learning Rate:** `[1e-3, 1e-6]` -> The learning rate for the classifier will be 10x the transformer learning rate, as described in the optimizer section.
- **Classifier Hidden Dimension:** `[64, 512]`
- **Dropout Rate:** `[0.1, 0.3]`
- **Weight Decay:** `[1e-4, 1e-6]`
- **Warmup Steps:** `[0.0, 0.1]` in % of total number of steps


### Training Strategy
For testing the model will be run with manually set hyperparameters. In a second stage the model will utilize optuna to automatically find the optimal hyperparameter combination.
- **Epochs:** A maximum of 100 epochs is set, with early stopping enabled. *This will be adjusted based on the runtime per epoch.*
- **Warmup Steps:** 0-10% warmup steps improved convergence in prior transformer projects during training. Will test with and without warmup.


### Metrics
- **Validation Accuracy:** To evaluate model performance across different hyperparameter configurations, I will use validation accuracy as the primary metric.
- **Confusion Matrix:** This will give a comprehensive view of true positives, true negatives, false positives, and false negatives, allowing me deeper insight into the model’s performance.


### Logging
Weights and Biases (WandB) is used for experiment tracking, logging metrics such as train and validation loss, accuracy, and confusion matrices.


this I want to change:
- warmup_steps should be defined in percent of total steps. i want to pass 0.1 for a 10% warmup step period.
- make model use warmup_steps and weight_decay
- 

In [64]:
class WandbLoggingCallback(pl.Callback):
    def __init__(self, log_interval: int = 10):
        self.log_interval = log_interval
        
        def on_train_batch_end(self, trainer, pl_module, outputs, batch, batch_idx):
            if (batch_idx + 1) % self.log_interval == 0:
                metrics = trainer.callback_metrics
                wandb.log({
                    "train_loss": metrics.get("train_loss", None),
                    "train_acc": metrics.get("train_acc", None),
                })
        
        def on_validation_epoch_end(self, trainer, pl_module):
            metrics = trainer.callback_metrics
            wandb.log({
                "val_loss": metrics.get("val_loss_epoch", None),
                "val_acc": metrics.get("val_acc_epoch", None),
            })
        
def format_run_name(hyperparams: dict) -> str:
    return "-".join([f"{key[:2]}_{val}" for key, val in hyperparams.items()])

def manual_training_run():
    hyperparameters = {
        "model_name": "bert-base-cased",
        "learning_rate": 1e-5,
        "hidden_dim": 256,
        "dropout_rate": 0.1,
        "weight_decay": 1e-5,
        "warmup_steps": 0.1,  # 10% warmup
        "epochs": 50,
        "batch_size": BATCH_SIZE,
    }
    
    # run_name = format_run_name(hyperparameters)
    group_name = "manual_testing"
    run_name = "testing_run"
    
    
    wandb_logger = WandbLogger(
        project="nlp-p4-pretrained_transformers",
        name=run_name,
        group=group_name,
    )
    
    data_module = BoolQDataModule(
        tokenizer_name=hyperparameters["model_name"], 
        batch_size=hyperparameters["batch_size"],
    )
    
    model = TransformerClassifier(
        model_name=hyperparameters["model_name"],
        learning_rate=hyperparameters["learning_rate"],
        hidden_dim=hyperparameters["hidden_dim"],
        dropout_rate=hyperparameters["dropout_rate"],
        warmup_steps=hyperparameters["warmup_steps"],
        weight_decay=hyperparameters["weight_decay"],
    )
    
    early_stopping = EarlyStopping(monitor="val_loss", patience=10, mode="min")
    checkpoint = ModelCheckpoint(monitor="val_acc", mode="max", save_top_k=1, filename=run_name)
    
    trainer = Trainer(
        max_epochs=hyperparameters["epochs"],
        callbacks=[early_stopping, checkpoint, WandbLoggingCallback()],
        accelerator="auto",
        devices=1,
        logger=wandb_logger,
    )

    trainer.fit(model, datamodule=data_module)

    wandb.finish()

In [54]:
manual_training_run()

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name             | Type                  | Params | Mode 
-------------------------------------------------------------------
0 | bert             | BertModel             | 108 M  | eval 
1 | classifier       | Sequential            | 197 K  | train
2 | loss_fn          | BCELoss               | 0      | train
3 | train_accuracy   | BinaryAccuracy        | 0      | train
4 | val_accuracy     | BinaryAccuracy        | 0      | train
5 | test_accuracy    | BinaryAccuracy        | 0      | train
6 | val_conf_matrix  | BinaryConfusionMatrix | 0      | train
7 | test_conf_matrix | BinaryConfusionMatrix | 0      | train
-------------------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
434.030   Total estimated model params size (MB)
12        Modules in train mode
228       Modules in eval mo

Training: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
# TODO: implement optuna here

## Evaluation
The percentage of yes answers in each data split is: Train; 62.64%, Val; 59.50%, Test;62.17%
Seeing how difficult it was in past projects to reach a much better accuracy than the baseline majority class I am setting my goal for the pretrained BERT model at 64% accuracy on the test set.


### Error Analysis
To understand why the model may fail on certain predictions, I will conduct an error analysis investigating weather miss classifications are related to the confidence score the model has in its predictions. Low confidence on correct answers or high confidence on wrong answers may indicate areas where the model is uncertain or overconfident.


### Confusion Matrix
After the validation step, a confusion matrix is computed to assess true positives, false positives, true negatives, and false negatives. This provides insights into the model's prediction performance.


## Planned Correctness Tests
- Visually checking for decreasing loss during training.
- Verifying predictions with a confusion matrix.

## Interpretation

My last project went decently well, beating the majority class accuracy of 62.17% on the test set. Before writing this interpretation i toyed around with the `bert-large-cased` model, implementing and running it as quickly as possible just to see what it could do. With 333 Million parameters in the transformer model I had to use a `batch_size` of 16 to not run out of memory. Giving it a single run over the weekend, with "looks about right" choice for hyperparameters, it managed to reach a test accuracy of 72.63% after over 23 hours of runtime. Impressed by this result I am setting my expectations for the properly implemented and fine-tuned `bert-base-cased` model to reach a test accuracy of 69%. Nice.